In [54]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import multiprocessing as mp
import sys
import time
import datetime as dt
from datetime import datetime
from collections import OrderedDict as od
import re
import os
import json


In [81]:
# df = pd.read_csv('/Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/src/ml_investing_wne/data/raw/crypto/Bitstamp_ETHUSD.csv')
df = pd.read_csv('/Users/i0495036/Documents/sandbox/ml_investing_wne/ml_investing_wne/src/ml_investing_wne/data/raw/crypto/Bitstamp_btcusd.csv')
df.head()

,q,p,s,t,d,side,buy_id,sell_id
0,0.001500,40383.25,btcusd,1623729486876,2021-06-15,0,1371727922049028,1371727921057793
1,0.002050,40383.26,btcusd,1623729486876,2021-06-15,0,1371727922049028,1371727920283649
2,0.185857,40375.12,btcusd,1623729517598,2021-06-15,0,1371728047882246,1371728022577152
3,0.001500,40389.65,btcusd,1623729532894,2021-06-15,0,1371728110534658,1371728110309376
4,0.001700,40389.66,btcusd,1623729532894,2021-06-15,0,1371728110534658,1371728103247872


In [68]:
df.describe()

,q,p,t,side,buy_id,sell_id
count,2.130073e+06,2.130073e+06,2.130073e+06,2.130073e+06,2.130073e+06,2.130073e+06
mean,3.988168e+00,1.446210e+03,1.612561e+12,4.884260e-01,1.325977e+15,1.325757e+15
std,1.037105e+01,2.987298e+02,2.119603e+09,4.998661e-01,8.679598e+12,1.900168e+13
min,1.000000e-08,7.162400e+02,1.609459e+12,0.000000e+00,1.312703e+15,7.369556e+08
25%,3.000000e-01,1.215220e+03,1.610551e+12,0.000000e+00,1.317749e+15,1.317724e+15
50%,1.289590e+00,1.438780e+03,1.612372e+12,0.000000e+00,1.325208e+15,1.325205e+15
75%,4.374654e+00,1.720000e+03,1.614163e+12,1.000000e+00,1.332538e+15,1.332542e+15
max,2.000000e+03,2.040620e+03,1.617062e+12,1.000000e+00,1.344419e+15,1.344419e+15


In [82]:
df['date'] = pd.to_datetime(df['t'],unit='ms')
df.head()

,q,p,s,t,d,side,buy_id,sell_id,date
0,0.001500,40383.25,btcusd,1623729486876,2021-06-15,0,1371727922049028,1371727921057793,2021-06-15 03:58:06.876
1,0.002050,40383.26,btcusd,1623729486876,2021-06-15,0,1371727922049028,1371727920283649,2021-06-15 03:58:06.876
2,0.185857,40375.12,btcusd,1623729517598,2021-06-15,0,1371728047882246,1371728022577152,2021-06-15 03:58:37.598
3,0.001500,40389.65,btcusd,1623729532894,2021-06-15,0,1371728110534658,1371728110309376,2021-06-15 03:58:52.894
4,0.001700,40389.66,btcusd,1623729532894,2021-06-15,0,1371728110534658,1371728103247872,2021-06-15 03:58:52.894


In [70]:

import numpy as np

# expects a numpy array with trades
# each trade is composed of: [time, price, quantity]
def generate_volumebars(trades, frequency=10):
    times = trades[:,0]
    prices = trades[:,1]
    volumes = trades[:,2]
    ans = np.zeros(shape=(len(prices), 6))
    candle_counter = 0
    vol = 0
    lasti = 0
    for i in range(len(prices)):
        vol += volumes[i]
        if vol >= frequency:
            ans[candle_counter][0] = times[i]                          # time
            ans[candle_counter][1] = prices[lasti]                     # open
            ans[candle_counter][2] = np.max(prices[lasti:i+1])         # high
            ans[candle_counter][3] = np.min(prices[lasti:i+1])         # low
            ans[candle_counter][4] = prices[i]                         # close
            ans[candle_counter][5] = np.sum(volumes[lasti:i+1])        # volume
            candle_counter += 1
            lasti = i+1
            vol = 0
    return ans[:candle_counter]

In [9]:
df['datepart'] = df['date'].dt.date

In [19]:
df_group = df.groupby('datepart')['q'].sum()
df_group.mean()

96535.09047195739

In [20]:
df_group

datepart
2021-01-01     65329.722823
2021-01-02     97776.328236
2021-01-03    245606.838081
2021-01-04    305975.548538
2021-01-05    291617.144051
                  ...      
2021-03-25     47979.714654
2021-03-26     51594.880197
2021-03-27     32393.114477
2021-03-28     21922.887577
2021-03-29     47197.860109
Name: q, Length: 88, dtype: float64

In [83]:
df['date'].min()

Timestamp('2019-04-03 13:08:02.802000')

In [84]:
df['date'].max()

Timestamp('2021-11-26 11:59:56.193000')

In [23]:
df.iloc[0:1000].to_csv('bitstamp.csv',sep=';', decimal=',')

In [28]:
df.head()

,q,p,s,t,d,side,buy_id,sell_id,datepart
date,,,,,,,,,
2021-01-01 11:28:43.281,1.000000,741.77,ethusd,1609500523281,2021-01-01,1,1313445995876352,1313446087163904,2021-01-01
2021-01-01 11:28:43.477,7.678646,741.55,ethusd,1609500523477,2021-01-01,1,1313446087430144,1313446087966720,2021-01-01
2021-01-01 11:28:43.477,11.657815,741.55,ethusd,1609500523477,2021-01-01,1,1313446087430145,1313446087966720,2021-01-01
2021-01-01 11:28:43.477,19.100000,741.54,ethusd,1609500523477,2021-01-01,1,1313446037651456,1313446087966720,2021-01-01
2021-01-01 11:28:43.477,16.852000,741.39,ethusd,1609500523477,2021-01-01,1,1313446082715648,1313446087966720,2021-01-01


In [24]:
def get_t_events(raw_price, threshold):
    """
    :param raw_price: (series) of close prices.
    :param threshold: (float) when the abs(change) is larger than the threshold, the
    function captures it as an event.
    :return: (datetime index vector) vector of datetimes when the events occurred. This is used later to sample.
    """
    print('Applying Symmetric CUSUM filter.')

    t_events = []
    s_pos = 0
    s_neg = 0

    # log returns
    diff = np.log(raw_price).diff().dropna()

    # Get event time stamps for the entire series
    for i in tqdm(diff.index[1:]):
        pos = float(s_pos + diff.loc[i])
        neg = float(s_neg + diff.loc[i])
        s_pos = max(0.0, pos)
        s_neg = min(0.0, neg)

        if s_neg < -threshold:
            s_neg = 0
            t_events.append(i)

        elif s_pos > threshold:
            s_pos = 0
            t_events.append(i)

    event_timestamps = pd.DatetimeIndex(t_events)
    return event_timestamps

In [33]:
def get_daily_vol(close, lookback=100):
    """
    :param close: (data frame) Closing prices
    :param lookback: (int) lookback period to compute volatility
    :return: (series) of daily volatility value
    """
    print('Calculating daily volatility for dynamic thresholds')
    
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0 > 0]
    df0 = (pd.Series(close.index[df0 - 1], index=close.index[close.shape[0] - df0.shape[0]:]))
        
    df0 = close.loc[df0.index] / close.loc[df0.values].values - 1  # daily returns
    df0 = df0.ewm(span=lookback).std()
    return df0

In [36]:
def get_t_events(raw_price, threshold):
    """
    :param raw_price: (series) of close prices.
    :param threshold: (float) when the abs(change) is larger than the threshold, the
    function captures it as an event.
    :return: (datetime index vector) vector of datetimes when the events occurred. This is used later to sample.
    """
    print('Applying Symmetric CUSUM filter.')

    t_events = []
    s_pos = 0
    s_neg = 0

    # log returns
    diff = np.log(raw_price).diff().dropna()

    # Get event time stamps for the entire series
    for i in tqdm(diff.index[1:]):
        pos = float(s_pos + diff.loc[i])
        neg = float(s_neg + diff.loc[i])
        s_pos = max(0.0, pos)
        s_neg = min(0.0, neg)

        if s_neg < -threshold:
            s_neg = 0
            t_events.append(i)

        elif s_pos > threshold:
            s_pos = 0
            t_events.append(i)

    event_timestamps = pd.DatetimeIndex(t_events)
    return event_timestamps

In [43]:
def add_vertical_barrier(t_events, close, num_days=1):
    """
    :param t_events: (series) series of events (symmetric CUSUM filter)
    :param close: (series) close prices
    :param num_days: (int) maximum number of days a trade can be active
    :return: (series) timestamps of vertical barriers
    """
    t1 = close.index.searchsorted(t_events + pd.Timedelta(days=num_days))
    t1 = t1[t1 < close.shape[0]]
    t1 = pd.Series(close.index[t1], index=t_events[:t1.shape[0]])  # NaNs at end
    return t1

In [50]:
def apply_pt_sl_on_t1(close, events, pt_sl, molecule):
    """
    :param close: (series) close prices
    :param events: (series) of indices that signify "events" 
    :param pt_sl: (array) element 0, indicates the profit taking level; 
                          element 1 is stop loss level
    :param molecule: (an array) a set of datetime index values for processing
    :return: (dataframe) timestamps at which each barrier was touched
    """
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_ = events.loc[molecule]
    out = events_[['t1']].copy(deep=True)
    if pt_sl[0] > 0:
        pt = pt_sl[0] * events_['trgt']
    else:
        pt = pd.Series(index=events.index)  # NaNs

    if pt_sl[1] > 0:
        sl = -pt_sl[1] * events_['trgt']
    else:
        sl = pd.Series(index=events.index)  # NaNs

    for loc, t1 in events_['t1'].fillna(close.index[-1]).iteritems():
        df0 = close[loc:t1]  # path prices
        df0 = (df0 / close[loc] - 1) * events_.at[loc, 'side']  # path returns
        out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()  # earliest stop loss
        out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()  # earliest profit taking

    return out

In [55]:
class MultiProcessingFunctions:
	""" This static functions in this class enable multi-processing"""
	def __init__(self):
		pass

	@staticmethod
	def lin_parts(num_atoms, num_threads):
		""" This function partitions a list of atoms in subsets (molecules) of equal size.
		An atom is a set of indivisible set of tasks.
		"""

		# partition of atoms with a single loop
		parts = np.linspace(0, num_atoms, min(num_threads, num_atoms) + 1)
		parts = np.ceil(parts).astype(int)
		return parts

	@staticmethod
	def nested_parts(num_atoms, num_threads, upper_triangle=False):
		""" This function enables parallelization of nested loops.
		"""
		# partition of atoms with an inner loop
		parts = []
		num_threads_ = min(num_threads, num_atoms)

		for num in range(num_threads_):
			part = 1 + 4 * (parts[-1] ** 2 + parts[-1] + num_atoms * (num_atoms + 1.) / num_threads_)
			part = (-1 + part ** .5) / 2.
			parts.append(part)

		parts = np.round(parts).astype(int)

		if upper_triangle:  # the first rows are heaviest
			parts = np.cumsum(np.diff(parts)[::-1])
			parts = np.append(np.array([0]), parts)
		return parts

	@staticmethod
	def mp_pandas_obj(func, pd_obj, num_threads=24, mp_batches=1, lin_mols=True, **kargs):
		"""	
		:param func: (string) function to be parallelized
		:param pd_obj: (vector) Element 0, is name of argument used to pass the molecule;
						Element 1, is the list of atoms to be grouped into a molecule
		:param num_threads: (int) number of threads
		:param mp_batches: (int) number of batches
		:param lin_mols: (bool) Tells if the method should use linear or nested partitioning
		:param kargs: (var args)
		:return: (data frame) of results
		"""

		if lin_mols:
			parts = MultiProcessingFunctions.lin_parts(len(pd_obj[1]), num_threads * mp_batches)
		else:
			parts = MultiProcessingFunctions.nested_parts(len(pd_obj[1]), num_threads * mp_batches)

		jobs = []
		for i in range(1, len(parts)):
			job = {pd_obj[0]: pd_obj[1][parts[i - 1]:parts[i]], 'func': func}
			job.update(kargs)
			jobs.append(job)

		if num_threads == 1:
			out = MultiProcessingFunctions.process_jobs_(jobs)
		else:
			out = MultiProcessingFunctions.process_jobs(jobs, num_threads=num_threads)

		if isinstance(out[0], pd.DataFrame):
			df0 = pd.DataFrame()
		elif isinstance(out[0], pd.Series):
			df0 = pd.Series()
		else:
			return out

		for i in out:
			df0 = df0.append(i)

		df0 = df0.sort_index()
		return df0

	@staticmethod
	def process_jobs_(jobs):
		""" Run jobs sequentially, for debugging """
		out = []
		for job in jobs:
			out_ = MultiProcessingFunctions.expand_call(job)
			out.append(out_)
		return out

	@staticmethod
	def expand_call(kargs):
		""" Expand the arguments of a callback function, kargs['func'] """
		func = kargs['func']
		del kargs['func']
		out = func(**kargs)
		return out

	@staticmethod
	def report_progress(job_num, num_jobs, time0, task):
		# Report progress as asynch jobs are completed

		msg = [float(job_num) / num_jobs, (time.time() - time0)/60.]
		msg.append(msg[1] * (1/msg[0] - 1))
		time_stamp = str(dt.datetime.fromtimestamp(time.time()))

		msg = time_stamp + ' ' + str(round(msg[0]*100, 2)) + '% '+task+' done after ' + \
			str(round(msg[1], 2)) + ' minutes. Remaining ' + str(round(msg[2], 2)) + ' minutes.'

		if job_num < num_jobs:
			sys.stderr.write(msg+'\r')
		else:
			sys.stderr.write(msg+'\n')

		return

	@staticmethod
	def process_jobs(jobs, task=None, num_threads=24):
		""" Run in parallel. jobs must contain a 'func' callback, for expand_call"""

		if task is None:
			task = jobs[0]['func'].__name__

		pool = mp.Pool(processes=num_threads)
		# outputs, out, time0 = pool.imap_unordered(MultiProcessingFunctions.expand_call,jobs),[],time.time()
		outputs = pool.imap_unordered(MultiProcessingFunctions.expand_call, jobs)
		out = []
		time0 = time.time()

		# Process asyn output, report progress
		for i, out_ in enumerate(outputs, 1):
			out.append(out_)
			MultiProcessingFunctions.report_progress(i, len(jobs), time0, task)

		pool.close()
		pool.join()  # this is needed to prevent memory leaks
		return out

In [49]:
def get_events(close, t_events, pt_sl, target, min_ret, num_threads, 
              vertical_barrier_times=False, side=None):
    """
    :param close: (series) Close prices
    :param t_events: (series) of t_events. 
                     These are timestamps that will seed every triple barrier.
    :param pt_sl: (2 element array) element 0, indicates the profit taking level; 
                  element 1 is stop loss level.
                  A non-negative float that sets the width of the two barriers. 
                  A 0 value means that the respective horizontal barrier will be disabled.
    :param target: (series) of values that are used (in conjunction with pt_sl)
                   to determine the width of the barrier.
    :param min_ret: (float) The minimum target return required for running a triple barrier search.
    :param num_threads: (int) The number of threads concurrently used by the function.
    :param vertical_barrier_times: (series) A pandas series with the timestamps of the vertical barriers.
    :param side: (series) Side of the bet (long/short) as decided by the primary model
    :return: (data frame) of events
            -events.index is event's starttime
            -events['t1'] is event's endtime
            -events['trgt'] is event's target
            -events['side'] (optional) implies the algo's position side
    """

    # 1) Get target
    target = target.loc[target.index.intersection(t_events)]
    target = target[target > min_ret]  # min_ret

    # 2) Get vertical barrier (max holding period)
    if vertical_barrier_times is False:
        vertical_barrier_times = pd.Series(pd.NaT, index=t_events)

    # 3) Form events object, apply stop loss on vertical barrier
    if side is None:
        side_ = pd.Series(1., index=target.index)
        pt_sl_ = [pt_sl[0], pt_sl[0]]
    else:
        side_ = side.loc[target.index]
        pt_sl_ = pt_sl[:2]

    events = pd.concat({'t1': vertical_barrier_times, 'trgt': target, 'side': side_},
                        axis=1)
    events = events.dropna(subset=['trgt'])

    # Apply Triple Barrier
    df0 = MultiProcessingFunctions.mp_pandas_obj(func=apply_pt_sl_on_t1,
                                                 pd_obj=('molecule', events.index),
                                                 num_threads=num_threads,
                                                 close=close,
                                                 events=events,
                                                 pt_sl=pt_sl_)

    events['t1'] = df0.dropna(how='all').min(axis=1)  # pd.min ignores nan

    if side is None:
        events = events.drop('side', axis=1)

    return events

In [72]:
df =df.set_index('date')
price = df['p'].resample('720min').agg({'open': 'first',
                                           'high': 'max',
                                           'low': 'min',
                                           'close': 'last'
                                           })
volume = df['q'].resample('720min').agg({'volume':'sum'})

df = price.join(volume)

In [58]:
# compute sides
df['side'] = np.nan
long_signals = (df['close'] <= df['lower'])
short_signals = (df['close'] >= df['upper'])
df.loc[long_signals, 'side'] = 1
df.loc[short_signals, 'side'] = -1

print(df.side.value_counts())

# Remove Look ahead biase by lagging the signal
df['side'] = df['side'].shift(1)

# Drop the NaN values from our data set
df.dropna(axis=0, how='any', inplace=True)  

KeyError: 'lower'

In [73]:
close = df['close']

In [74]:
daily_vol = get_daily_vol(close=close, lookback=50)

Calculating daily volatility for dynamic thresholds


In [75]:
daily_vol

date
2021-01-02 12:00:00         NaN
2021-01-03 00:00:00    0.062692
2021-01-03 12:00:00    0.149714
2021-01-04 00:00:00    0.128269
2021-01-04 12:00:00    0.112950
                         ...   
2021-03-27 12:00:00    0.054111
2021-03-28 00:00:00    0.053039
2021-03-28 12:00:00    0.051989
2021-03-29 00:00:00    0.051255
2021-03-29 12:00:00    0.051840
Freq: 720T, Name: close, Length: 173, dtype: float64

In [76]:
# creating our event triggers using the CUSUM filter 
cusum_events = get_t_events(close, threshold=daily_vol.mean()*0.1)

Applying Symmetric CUSUM filter.


100%|██████████| 174/174 [00:00<00:00, 12425.03it/s]


In [77]:
cusum_events

DatetimeIndex(['2021-01-02 12:00:00', '2021-01-03 00:00:00',
               '2021-01-03 12:00:00', '2021-01-04 12:00:00',
               '2021-01-05 12:00:00', '2021-01-06 00:00:00',
               '2021-01-06 12:00:00', '2021-01-07 12:00:00',
               '2021-01-08 00:00:00', '2021-01-08 12:00:00',
               ...
               '2021-03-23 12:00:00', '2021-03-24 00:00:00',
               '2021-03-24 12:00:00', '2021-03-26 00:00:00',
               '2021-03-26 12:00:00', '2021-03-27 00:00:00',
               '2021-03-27 12:00:00', '2021-03-28 12:00:00',
               '2021-03-29 00:00:00', '2021-03-29 12:00:00'],
              dtype='datetime64[ns]', length=141, freq=None)

In [78]:
# adding vertical barriers with a half day expiration window
vertical_barriers = add_vertical_barrier(t_events=cusum_events,
                                         close=close, num_days=0.5)

In [79]:
vertical_barriers

2021-01-02 12:00:00   2021-01-03 00:00:00
2021-01-03 00:00:00   2021-01-03 12:00:00
2021-01-03 12:00:00   2021-01-04 00:00:00
2021-01-04 12:00:00   2021-01-05 00:00:00
2021-01-05 12:00:00   2021-01-06 00:00:00
                              ...        
2021-03-26 12:00:00   2021-03-27 00:00:00
2021-03-27 00:00:00   2021-03-27 12:00:00
2021-03-27 12:00:00   2021-03-28 00:00:00
2021-03-28 12:00:00   2021-03-29 00:00:00
2021-03-29 00:00:00   2021-03-29 12:00:00
Name: date, Length: 140, dtype: datetime64[ns]

In [80]:
pt_sl = [1, 2] # setting profit-take and stop-loss at 1% and 2%
min_ret = 0.0005 # setting a minimum return of 0.05%

triple_barrier_events = get_events(close=close,
                                  t_events=cusum_events,
                                  pt_sl=pt_sl,
                                  target=daily_vol,
                                  min_ret=min_ret,
                                  num_threads=2,
                                  vertical_barrier_times=vertical_barriers)

Process SpawnPoolWorker-10:
Process SpawnPoolWorker-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/i0495036/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/i0495036/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/i0495036/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/i0495036/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/i0495036/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/i0495036/.pyenv/versions/3.8.12/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
  File "/Users/i0495036/.pyenv/versions/3.8

KeyboardInterrupt: 